In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool
import os

# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)





In [2]:
# # 步骤 2：配置您所使用的 LLM。
# llm_cfg = {
#     # 使用 DashScope 提供的模型服务：
#     'model': 'qwen-max',
#     'model_server': 'dashscope',
#     'api_key': os.getenv('DASHSCOPE_API_KEY'),
#     'generate_cfg': {
#         'top_p': 0.8
#     }
# }

# llm_cfg = {
#     # 使用 DashScope 提供的模型服务：
#     'model': 'deepseek-v3',
#     'model_server': 'https://dashscope.aliyuncs.com/compatible-mode/v1',
#     'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
#     'generate_cfg': {
#         'top_p': 0.8
#     }
# }



# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-max',
    'model_server': 'dashscope',
    # 'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    'api_key': os.getenv('DASHSCOPE_API_KEY'),
    # 如果这里没有设置 'api_key'，它将读取 `DASHSCOPE_API_KEY` 环境变量。

    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    # 'model': 'Qwen2-7B-Chat',
    # 'model_server': 'http://localhost:8000/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',
    # （可选） LLM 的超参数：
    'generate_cfg': {
        'top_p': 0.8
    }
}


In [3]:
# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先绘制一幅图像，图像中的文字尽量用中文，不要有乱码，
- 得到图像的url，
- 然后运行代码`request.get`以下载该图像的url，
- 最后从给定的文档中选择一个图像操作进行图像处理。
- 用 `plt.show()` 展示图像。
你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
import os
# 获取文件夹下所有文件
file_dir = os.path.join('./', 'docs')
files = []
if os.path.exists(file_dir):
    # 遍历目录下的所有文件
    for file in os.listdir(file_dir):
        file_path = os.path.join(file_dir, file)
        if os.path.isfile(file_path):  # 确保是文件而不是目录
            files.append(file_path)
print('files=', files)

bot = Assistant(llm=llm_cfg,   
                system_message=system_instruction,
                function_list=tools,
                files=files)

#另外一种使用本地下载的Model的方式
# from modelscope import snapshot_download
# model_dir = snapshot_download("qwen/Qwen-7B-Chat")
# bot = Assistant(
#     model="Qwen/Qwen-7B-Chat",  # 模型路径或 Hugging Face ID
#     tools=["code_interpreter", "web_search"],  # 可选工具
#     system_message="你是一个智能助手，可以回答问题并执行任务。"
# )


files= ['./docs\\1-平安商业综合责任保险（亚马逊）.txt', './docs\\2-雇主责任险.txt', './docs\\3-平安企业团体综合意外险.txt', './docs\\4-雇主安心保.txt', './docs\\5-施工保.txt', './docs\\6-财产一切险.txt', './docs\\7-平安装修保.txt', './docs\\平安产险交通出行意外伤害保险（互联网版）产品说明.pdf', './docs\\平安产险交通工具意外伤害保险（互联网版）条款.pdf', './docs\\平安企业团体综合意外险(互联网版)适用条款.pdf', './docs\\平安商业综合责任保险（亚马逊）.pdf', './docs\\平安境内紧急医疗救援服务条款.pdf', './docs\\平安附加疾病身故保险条款.pdf']


In [ ]:

# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
while True:
    # 例如，输入请求 "绘制一只狗并将其旋转 90 度"。
    query = input('用户请求: ')
    if query == '-1':
        break
#    query = "介绍下雇主责任险"
    # 将用户请求添加到聊天历史。
    messages.append({'role': 'user', 'content': query})
    # response = []
    current_index = 0
    response_list= list(bot.run(messages=messages)) #将生成器转换为列表
#    for response in bot.run(messages=messages):
    for response in response_list :
        if current_index == 0:
            # 尝试获取并打印召回的文档内容
            if hasattr(bot, 'retriever') and bot.retriever:
                print("\n===== 召回的文档内容 =====")
                retrieved_docs = bot.retriever.retrieve(query)
                if retrieved_docs:
                    for i, doc in enumerate(retrieved_docs):
                        print(f"\n文档片段 {i+1}:")
                        print(f"内容: {doc.page_content}")
                        print(f"元数据: {doc.metadata}")
                else:
                    print("没有召回任何文档内容")
                print("===========================\n")
            #break
    
        current_response = response[0]['content'][current_index:]
        current_index = len(response[0]['content'])
        print(current_response, end='')

   
    last_response= response_list[-1]  #取出最后生成的完整的response
    contents = [str(item['content']) for item in last_response if item.get('content')]#取出response的各个字典元素中所有content的值
    combined_content = ''.join(contents) #将这些值拼成完整的回答。
    print(combined_content)
    # 将机器人的回应添加到聊天历史。
    messages.extend(last_response)

用户请求:  请根据文档绘制一张保险业务的关联图


2025-04-26 18:24:05,172 - memory.py - 116 - INFO - {"keywords_zh": ["文档", "绘制", "保险业务", "关联图"], "keywords_en": ["document", "draw", "insurance business", "relationship diagram", "chart"], "text": "请根据文档绘制一张保险业务的关联图"}
2025-04-26 18:24:05,786 - simple_doc_parser.py - 411 - INFO - Read parsed ./docs\1-平安商业综合责任保险（亚马逊）.txt from cache.
2025-04-26 18:24:05,787 - doc_parser.py - 108 - INFO - Start chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt)...
2025-04-26 18:24:05,788 - doc_parser.py - 126 - INFO - Finished chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt). Time spent: 0.0 seconds.
2025-04-26 18:24:05,792 - simple_doc_parser.py - 411 - INFO - Read parsed ./docs\2-雇主责任险.txt from cache.
2025-04-26 18:24:05,794 - doc_parser.py - 108 - INFO - Start chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt)...
2025-04-26 18:24:05,794 - doc_parser.py - 126 - INFO - Finished chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt). Time spent: 0.0 seconds.
2025-04-26 18:24:05,797 - simple_doc_parser.py 

{image_url: "https://image.pollinations.ai/prompt/%E7%BB%98%E5%88%B6%E4%B8%80%E5%BC%A0%E4%BF%9D%E9%99%A9%E4%B8%9A%E5%8A%A1%E5%85%B3%E8%81%94%E5%9B%BE%EF%BC%8C%E5%8C%85%E6%8B%AC%E5%B9%B3%E5%AE%89%E5%95%86%E4%B8%9A%E7%BB%BC%E5%90%88%E8%B4%A3%E4%BB%BB%E4%BF%9D%E9%99%A9%EF%BC%88%E4%BA%9A%E9%A9%AC%E9%80%8A%EF%BC%89%E3%80%81%E9%9B%87%E4%B8%BB%E8%B4%A3%E4%BB%BB%E9%99%A9%E3%80%81%E5%B9%B3%E5%AE%89%E4%BC%81%E4%B8%9A%E5%9B%A2%E4%BD%93%E7%BB%BC%E5%90%88%E6%84%8F%E5%A4%96%E9%99%A9%E3%80%81%E6%96%BD%E5%B7%A5%E4%BF%9D%E5%92%8C%E5%B9%B3%E5%AE%89%E8%A3%85%E4%BF%AE%E4%BF%9D%E3%80%82%E8%BF%99%E4%BA%9B%E4%BF%9D%E9%99%A9%E4%BA%A7%E5%93%81%E5%9B%B4%E7%BB%95%E7%9D%80%E4%BC%81%E4%B8%9A%E5%92%8C%E4%B8%AA%E4%BA%BA%E6%8F%90%E4%BE%9B%E4%B8%8D%E5%90%8C%E6%96%B9%E9%9D%A2%E7%9A%84%E4%BF%9D%E9%9A%9C%E3%80%82%E4%BE%8B%E5%A6%82%EF%BC%8C%E5%B9%B3%E5%AE%89%E5%95%86%E4%B8%9A%E7%BB%BC%E5%90%88%E8%B4%A3%E4%BB%BB%E4%BF%9D%E9%99%A9%EF%BC%88%E4%BA%9A%E9%A9%AC%E9%80%8A%EF%BC%89%E4%B8%BA%E5%9C%A8%E4%BA%9A%E9%A9%AC%E9%80%8A%E5%B

用户请求:  请帮我绘制一张”孤帆远隐碧空尽“的图


2025-04-26 18:29:03,028 - memory.py - 116 - INFO - {"keywords_zh": ["绘制", "图", "孤帆", "远隐", "碧空"], "keywords_en": ["draw", "image", "solitary sail", "distant", "blue sky"], "text": "请帮我绘制一张”孤帆远隐碧空尽“的图"}
2025-04-26 18:29:03,032 - simple_doc_parser.py - 411 - INFO - Read parsed ./docs\1-平安商业综合责任保险（亚马逊）.txt from cache.
2025-04-26 18:29:03,034 - doc_parser.py - 108 - INFO - Start chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt)...
2025-04-26 18:29:03,035 - doc_parser.py - 126 - INFO - Finished chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt). Time spent: 0.0 seconds.
2025-04-26 18:29:03,040 - simple_doc_parser.py - 411 - INFO - Read parsed ./docs\2-雇主责任险.txt from cache.
2025-04-26 18:29:03,042 - doc_parser.py - 108 - INFO - Start chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt)...
2025-04-26 18:29:03,045 - doc_parser.py - 126 - INFO - Finished chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt). Time spent: 0.0009999275207519531 seconds.
2025-04-26 18:29:03,051 - simple_doc_parser.

{image_url: "https://image.pollinations.ai/prompt/%E5%AD%A4%E5%B8%86%E8%BF%9C%E9%9A%90%E7%A2%A7%E7%A9%BA%E5%B0%BD"}这是根据您的描述“孤帆远隐碧空尽”生成的图像：

![孤帆远隐碧空尽](https://image.pollinations.ai/prompt/%E5%AD%A4%E5%B8%86%E8%BF%9C%E9%9A%90%E7%A2%A7%E7%A9%BA%E5%B0%BD)

这张图描绘了一艘孤独的小船在广阔的天空下逐渐远去，直至消失在天际。希望这幅画能够符合您的期待。

如果您需要进一步的帮助或有其他要求，请告诉我！
